<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/main/growth_comp_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

# Filter and select relevant columns
data = pwt90.loc[pwt90['country'].isin(["Japan", "Germany", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 2000) & (data['year'] <= 2015)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['dk'] = (grouped_data['k_pc'].diff() * 100)  # Growth rate of capital per capita
data['dsolow'] = data['g'] - data['a'] * data['dk']  # Solow residual

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean',
                                       'dsolow': 'mean',
                                       'a': "mean",
                                       'dk': "mean"})

# Calculate additional summary statistics
summary['tfp.growth'] = summary['dsolow']
summary['capitakl deepning'] = summary['a']*summary['dk']
summary['tfp.share'] = summary['dsolow'] / summary['g']
summary['capital.share'] = summary['capitakl deepning'] / summary['g']

# Print output
print(summary)

                    g    dsolow         a        dk  tfp.growth  \
countrycode                                                       
DEU          0.595897  0.232704  0.384552  0.967537    0.232704   
JPN          0.752177  0.139220  0.392263  1.588331    0.139220   
USA          1.256881  0.649874  0.387017  1.590940    0.649874   

             capitakl deepning  tfp.share  capital.share  
countrycode                                               
DEU                   0.372068   0.390511       0.624383  
JPN                   0.623043   0.185090       0.828320  
USA                   0.615721   0.517053       0.489880  
